<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/LLAMAIndex_Chatbot_Multiple_index_yearly_data_build_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
!pip -q install llama-index llama-index-readers-file openai llama-index-embeddings-openai unstructured

In [94]:
from google.colab import userdata
api_key=userdata.get('openai')

In [95]:
import os
os.environ['OPENAI_API_KEY']=api_key

In [96]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [97]:
!wget "https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?d1=1"

--2024-04-29 18:35:59--  https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?d1=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/948jr9cfs7fgj99/UBER.zip [following]
--2024-04-29 18:36:00--  https://www.dropbox.com/s/raw/948jr9cfs7fgj99/UBER.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1cdcba13c369086759e91ccc91.dl.dropboxusercontent.com/cd/0/inline/CR9Xm2437m4Wv3paIZM_jd-l-u6zDsQHBzKCO6_vJ-vfyFpDT8DHc-NQQWf7Pg7UpzzXDNnpEA0JiEURO0KuWWoS8GGBhNtFBvYGvhQe_InXqTrmWzajHFDN2AhB1kJnzTw/file# [following]
--2024-04-29 18:36:00--  https://uc1cdcba13c369086759e91ccc91.dl.dropboxusercontent.com/cd/0/inline/CR9Xm2437m4Wv3paIZM_jd-l-u6zDsQHBzKCO6_vJ-vfyFpDT8DHc-NQQWf7Pg7UpzzXDNnpEA0JiEURO0KuWWoS8GGBhNtFBvYGvhQe_InXqTrmWzajHFDN2AhB1kJnzT

In [98]:
#!unzip /content/data/UBER.zip?d1=1

Archive:  /content/data/UBER.zip?d1=1
replace UBER/UBER_2021.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [121]:
!pip -q  install llama-index-question-gen-openai

In [122]:
from llama_index.readers.file import UnstructuredReader

In [123]:
from pathlib import Path

In [124]:
years=[2019, 2020, 2021, 2022]

In [125]:
import nltk
nltk.download('averaged_perceptron_tagger')
loader=UnstructuredReader()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [103]:
all_doc=[]
doc_set={}
for year in years:
  year_doc=loader.load_data(file=Path(f"/content/UBER/UBER_{year}.html"))
  for doc in year_doc:
    doc.metadata={"year":year}
  doc_set[year]=year_doc
  all_doc.extend(year_doc)

In [ ]:
doc_set[2019]

In [105]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

In [106]:
from llama_index.core import Settings

In [126]:
Settings.chunk_size=512
Settings.chunk_overlap=64
Settings.model=OpenAI(model='gpt-3.5-turbo')
Settings.embedding=OpenAIEmbedding(model='text-embedding-3-small')

In [171]:
#yearly creating the index
#you can create multiple index (multi-files)
index_set={}
for year in years:
  sc=StorageContext.from_defaults()
  index=VectorStoreIndex.from_documents(
      doc_set[year],
      storage_context=sc
  )
  index_set[year]=index
  sc.persist(persist_dir=f'/content/Storage/{year}')

In [172]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [173]:
from llama_index.core.query_engine import SubQuestionQueryEngine

In [174]:
individual_query_engine_tools= [
    QueryEngineTool(
    #every year is a individual query engine
    query_engine=index_set[year].as_query_engine(),
    metadata=ToolMetadata(
        name=f"vector_index_{year}",
        description=(
            "useful for when you want to answer query about the "
            f" {year} for uber data"
        )
    )
)
    for year in years
 ]

In [175]:
query_engine=SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
)


In [206]:
# this is for entire data
query_engine_tool=QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="query_engine",
        description=(
           "useful for when you want to answer query that require analysing"
           "multiple documents"
        ),
    ),
)


In [207]:
individual_query_engine_tools

In [208]:
tools=individual_query_engine_tools + [query_engine_tool]

In [209]:
from llama_index.agent.openai import OpenAIAgent
agent=OpenAIAgent.from_tools(tools,verbose=True)


In [210]:
response=agent.chat('Hi')
print(str(response))

Added user message to memory: Hi
Hello! How can I assist you today?


In [213]:
response=agent.chat('What were some risk factors in 2020 with Uber?')
print(str(response))

Added user message to memory: What were some risk factors in 2020 with Uber?
=== Calling Function ===
Calling function: vector_index_2020 with args: {"input": "Uber risk factors"}
Got output: Uber faces various risk factors including potential liability claims related to the acts of drivers or platform users, challenges with insurance coverage compliance, regulatory scrutiny on pricing models, incidents of criminal activity involving drivers or users, and the need to invest in new offerings and technologies which may not yield expected benefits. These risks could lead to financial costs, negative publicity, regulatory actions, and impact on business operations and results.

=== Calling Function ===
Calling function: vector_index_2021 with args: {"input": "Uber risk factors"}
Got output: Uber faces various risk factors that could potentially impact its business, financial condition, operating results, and prospects. These risks include adverse effects from the COVID-19 pandemic and rela

In [214]:
response=agent.chat('What were some good factors in 2020 with Uber?')
print(str(response))

Added user message to memory: What were some good factors in 2020 with Uber?
=== Calling Function ===
Calling function: vector_index_2020 with args: {"input":"Uber positive factors"}
Got output: Uber offers accessible, flexible, and independent work opportunities that have been positively received by a significant percentage of their drivers. They have implemented innovative ways to address issues related to benefits and protections for independent workers, such as partnering with insurance companies to provide protections and benefits. Additionally, Uber has taken steps to improve the earnings potential of drivers by developing technology to help them make informed choices about when and where to drive. Furthermore, Uber has partnered with educational institutions to offer eligible drivers access to degree programs and professional courses, showing a commitment to driver progression. Uber's advocacy for legislative reform to modernize the social safety net and their engagement with dr

In [215]:
#testing memory
response=agent.chat('I didnt get those risk factors can you explain me 3 and 4? ')
print(str(response))

Added user message to memory: I didnt get those risk factors can you explain me 3 and 4? 
Certainly! Here are explanations for risk factors 3 and 4 associated with Uber in 2020:

3. **Intense Competition in the Mobility, Delivery, and Logistics Industries:**
   - Uber faced significant competition in the mobility, delivery, and logistics sectors in 2020. Competitors in these industries offered similar services and solutions, leading to pricing pressures and the need for continuous innovation to stay competitive. Intense competition could impact Uber's market share, pricing strategies, and profitability.

4. **Significant Losses Incurred Since Inception:**
   - Uber has experienced substantial losses since its inception due to various factors such as high operating costs, regulatory challenges, and investments in new technologies and markets. These losses could continue to impact Uber's financial performance, cash flow, and ability to achieve profitability in the future. Investors and s